In [1]:
import requests # http requests
from bs4 import BeautifulSoup # Webscrape
import pandas as pd     # DF
import csv
import time
header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'}
url = f'https://www.pap.fr/annonce/vente-appartements-paris-75-g439-studio' #JUST PUT HERE YOUR URL AFTER SEARCH ON PAP'S
from urllib.request import Request, urlopen
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
driver= webdriver.Chrome(ChromeDriverManager().install())
import numpy
import string
import re

[WDM] - Downloading: 100%|██████████| 6.46M/6.46M [00:00<00:00, 11.5MB/s]


In [26]:
def extractpaps(taille):
    #On recupére l'url sur sln et on execute un scrollbar en JS 
    driver.get(url)
    time.sleep(5)
    for i in range (0,taille):# 15 -240   30 - 345 1 environ = à 15 annonces dont 12 annonces de logement
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(3)  
    html1= BeautifulSoup(driver.page_source,"html.parser")
    Items = html1.find_all("div", class_="search-list-item-alt") # Recherches des soups par div et class 
    
    
    parameters = ['Titre','Prix','Descriptions','Taille','m^2']
    df = pd.DataFrame(columns=parameters)
    i=0
    #INDEXATION
    for item in Items:
        i = i + 1
        Price= item.find('span', class_="item-price")
        Titre= item.find('span', class_="h1")
        taille = item.find_all('li')
        Desc= item.find('p', class_="item-description")
        l=[]
        l.append(Titre.get_text().strip())
        l.append(Price.get_text().strip()) 
        l.append(Desc.get_text().strip())
        for e in taille:
            l.append(e.get_text().strip())
        if len(l) == 5 :
            df.loc[i]=l
        if len(l)== 4 :
            l.append('None')
            df.loc[i]=l
        if len(l) == 3 :
            l.append('None')
            l.append('None')
            df.loc[i]=l
    #EXCEL WRIT        
    exelwrit = pd.ExcelWriter('output.xlsx')   
    df.to_excel(
        exelwrit,  index=True, index_label=None, startrow=0, startcol=0,)

    exelwrit.close()
    print("Vous trouverez un fichier output.xlsx comprenant tout ce qui est dans DATA, toutes les annonces extraites de votre recherche ")
    return df

In [18]:
data = extractpaps(30)

In [19]:
data # Lire DATA, vous trouverez les data entieres dans output.xlsx

,Titre,Prix,Descriptions,Taille,m^2
1,Paris 16E (75116),500.000 €,Très beau studio de 33 m² dans une résidence l...,1 pièce,33.00 m2
2,Paris 8E (75008),175.000 €,"Rue de Saint Petersbourg, à 200 mètres du métr...",1 pièce,15.00 m2
3,Paris 13E,260.000 €,**10 villa d'Est. Tour Atlas Paris 13ème**\n**...,1 pièce,34.00 m2
4,Lutte anti-fraude,PAP contrôle toutes les annonces,"Avant de publier une annonce, nos équipes effe...",None,None
5,Paris 19E (75019),249.500 €,"Dans rue calme, studio au 4 ème étage d'un imm...",1 pièce,23.00 m2
...,...,...,...,...,...
341,Paris 14E (75014),112.000 €,"Métro Pernety et nombreux autobus à 5min, proc...",1 pièce,9.85 m2
342,Et si vous étiez éligible au logement interméd...,,Utilisez notre simulateur pour accéder à des o...,None,None
343,Paris 13E,260.000 €,**10 villa d'Est. Tour Atlas Paris 13ème**\n**...,1 pièce,34.00 m2
344,Paris 15E (75015),242.000 €,"**SOUS COMPROMIS**\n\n**Très bien situé, au se...",1 pièce,22.00 m2


In [20]:
def Yuushi1mean(data):
    
    PRICE = data.iloc[:,1]
    
    # transformer en int 
    prix = list()
    for e in PRICE :
        nombres = re.findall("\d+", e) # recherche tous les décimals
        if len(nombres) < 2 :
            pricee = 0
            prix.append(0)
        else :
            pricee= nombres[0] + nombres[1]
            prix.append(int(pricee))
        #suppression des zeroooes
        for e in prix:
            if e == 0:
                prix.remove(e)
   # print(prix) # Les prix des logements valides 
    print('Nombre de prix de logement valide :')
    print(len(prix)) #Nombre de logement 
    print ('Prix moyen de votre recherche :')
    return numpy.mean(prix) #moyenne du prix de votre recherche 
        
    

In [21]:
Yuushi1mean(data)

Nombre de prix de logement valide :
263
Prix moyen de votre recherche :


253264.95057034222

In [24]:
def Yuushi1m2mean():
    GRLIST = data.iloc[:,4]
    
    # transformer en int 
    m2 = list()
    for e in GRLIST :
        nombres = re.findall("\d+", e) # recherche tous les décimals
        if len(nombres) < 1 :
            m2.append(0)
        else :
            metr= nombres[0] 
            m2.append(int(metr))
        
        #suppr des zerooso
        for e in m2:
            if e ==0: 
                m2.remove(e)
   # print (m2)
    print('Nombre de m**2 de logement valide : (pub et logement sans taille non inclus dans le compte)') 
    print (len(m2))
    print ('Taille moyenne de votre recherche en m**2 :')
    return numpy.mean(m2) 

In [25]:
Yuushi1m2mean()

Nombre de m**2 de logement valide : (pub et logement sans taille non inclus dans le compte)
260
Taille moyenne de votre recherche en m**2 :


20.25